<a href="https://colab.research.google.com/github/imabari/covid19-data/blob/master/ehime/ehime_yousei_weeks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
# オープンデータ読み込み
df = pd.read_csv(
    "https://www.pref.ehime.jp/opendata-catalog/dataset/2174/resource/7073/380008_ehime_covid19_test_people.csv",
    index_col=0,
    parse_dates=True,
)

In [3]:
df

,全国地方公共団体コード,都道府県名,市区町村名,検査実施_人数,陽性者_人数,備考
実施_年月日,,,,,,
2020-02-01,380008,愛媛県,NaN,0,0,NaN
2020-02-02,380008,愛媛県,NaN,0,0,NaN
2020-02-03,380008,愛媛県,NaN,1,0,NaN
2020-02-04,380008,愛媛県,NaN,0,0,NaN
2020-02-05,380008,愛媛県,NaN,0,0,NaN
...,...,...,...,...,...,...
2021-04-15,380008,愛媛県,NaN,2776,45,NaN
2021-04-16,380008,愛媛県,NaN,1231,43,NaN
2021-04-17,380008,愛媛県,NaN,1285,48,NaN


In [4]:
# 公表日のため、確定日+1日
df.index = df.index + pd.Timedelta(days=1)

In [5]:
df_date = df.index.isocalendar()

# 週追加
df_date["weekday"] = df_date.index.weekday
df_date.drop("day", axis=1, inplace=True)

In [6]:
# 結合
df1 = df.join(df_date)

In [7]:
# 曜日別に並び替え
pv = (
    df1.pivot(index=["year", "week"], columns="weekday", values="陽性者_人数")
    .astype("Int64")
    .set_axis(["月", "火", "水", "木", "金", "土", "日"], axis=1).reset_index(drop=True)
)

In [8]:
pv

,月,火,水,木,金,土,日
0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0
1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...
60,3,23,24,59,39,19,27
61,24,30,33,30,28,34,29
62,32,43,34,31,27,33,24
63,17,19,30,43,45,43,48


In [9]:
# 週計計算
df2 = df["陽性者_人数"].resample("W").sum().reset_index()
df2.rename(columns={"陽性者_人数": "計", "実施_年月日": "終了"}, inplace=True)

In [10]:
# 開始追加
df2["開始"] = df2["終了"] - pd.Timedelta(days=6)

In [11]:
# 結合、列並び替え
df3 = df2.join(pv).reindex(columns=["開始", "終了", "月", "火", "水", "木", "金", "土", "日", "計"])

In [12]:
# CSV変換
df3.to_csv("yousei.csv", encoding="utf_8_sig", index=False)